In [22]:
import cv2
import mss
import numpy as np
from config import cfg
import os
from PIL import Image
from run import log_prices

## Coordinates and offsets:

`2560 * 1440px, windowed full-screen`:
- Sub-file : digits\QHD\
- Digit size (px): 9 * 14 
- Distance between digits (px, left to right) : 2, 1, 7, 1, 2, 6, 2, 1
- Vertical gap (px, top to bottom): 62, 61,  62, 61, 62, 62, 61, 62, 61, 62, 62, 61, 62
- Initial top-right first potential digit coordinates (px, X,Y) : 1457, 291
- Width and Height of one price box (px, W, H) : 103, 14 (runtime tests needed to see what box division is most efficient)


In [23]:
#USE WINDOWED FULL-SCREEN (Windows Bar at default size below) AND USE A SUPPORTED RESOLUTION TAG
# Currently supported : QHD (2560*1440)
# Enter a tag string for the value of DISPLAY_SIZE, (QHD, HD, FULL_HD, ULTRA_HD)
DISPLAY_SIZE = 'QHD_ROG'

In [24]:
config = cfg[DISPLAY_SIZE]
digit_gap = config['digit_gap']
price_gap = config['price_gap']
pixel_width = config['pixel_width']
pixel_height = config['pixel_height']
initial_X, initial_Y = config['initial_X_Y']
box_width, box_height = config['capture_box']
scroll_clicks = config['scroll_clicks']

print(price_gap)

[60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60]


In [25]:
digits = [cv2.imread(os.path.join('digits',DISPLAY_SIZE,str(i) + '.png')) for i in range(10)]

In [26]:
def extract_number(image):
    number = 0
    calc_gap = 0
    for i, gap in enumerate(digit_gap):
        calc_gap += gap
        cropped = image.crop((calc_gap + pixel_width * i, 0, calc_gap + pixel_width * (i+1), 14))
        res = [cv2.matchTemplate(np.array(cropped), digit, cv2.TM_CCOEFF_NORMED)[0] for digit in digits]
        if max(res) > 0.4:
            number = 10*number + res.index(max(res))
        if max(res) > 0 and max(res) < 0.55 and res.index(max(res)) == 1:
            print(max(res))
            cropped.show()
    return number

In [27]:
text = open(os.path.join('clean_data', 'resources.txt'), 'r', encoding='utf-8').read()
text = [line for line in text.splitlines() if line]
len(text[14:]) % 12

9

In [28]:
import pyautogui

with mss.mss() as sct:
    numbers = []    
    #Initial 2
    item1 = sct.grab({'top': initial_Y, 'left': initial_X, 'width': box_width, 'height':box_height})
    img = Image.frombytes("RGB", item1.size, item1.bgra, "raw", "BGRX")
    numbers.append(extract_number(img))
    item2 = sct.grab({'top': initial_Y+price_gap[0], 'left': initial_X, 'width': box_width, 'height':box_height})
    img2 = Image.frombytes("RGB", item2.size, item2.bgra, "raw", "BGRX")
    numbers.append(extract_number(img2))
    iterations = len(text[14:]) // 12
    remainder = len(text[14:]) % 12
    pyautogui.sleep(2)
    for i in range(12):
        item = sct.grab({'top': initial_Y+sum(price_gap[:i+2]), 'left': initial_X, 'width': box_width, 'height':box_height})
        numbers.append(extract_number(Image.frombytes("RGB", item.size, item.bgra, "raw", "BGRX")))
    for _ in range(iterations):
        pyautogui.scroll(scroll_clicks)
        for i in range(12):
            item = sct.grab({'top': initial_Y+sum(price_gap[:i+2]), 'left': initial_X, 'width': box_width, 'height':box_height})
            numbers.append(extract_number(Image.frombytes("RGB", item.size, item.bgra, "raw", "BGRX")))
    if remainder:
        pyautogui.scroll(scroll_clicks)
    for i in range(remainder-1, -1, -1):
        if i :
            item = sct.grab({'top': initial_Y+sum(price_gap[:-i]), 'left': initial_X, 'width': box_width, 'height':box_height})
        else :
            item = sct.grab({'top': initial_Y+sum(price_gap[:]), 'left': initial_X, 'width': box_width, 'height':box_height})
        numbers.append(extract_number(Image.frombytes("RGB", item.size, item.bgra, "raw", "BGRX")))
    log_prices(text, numbers, 'resource')